# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 9 2023 9:09AM,254527,16,EMERSA-9009426,"Emersa Waterbox, LLC",Released
1,Jan 9 2023 9:06AM,254531,10,Clear-8998803,"ClearSpec, LLC",Released
2,Jan 9 2023 9:06AM,254531,10,Clear-8998804,"ClearSpec, LLC",Released
3,Jan 9 2023 9:03AM,254530,10,DNMD-30929,Emerginnova,Released
4,Jan 9 2023 9:03AM,254530,10,Enova-11176,Emerginnova,Released
5,Jan 9 2023 9:02AM,254528,10,EYE43018,Eye Pharma Inc,Released
6,Jan 9 2023 9:02AM,254520,10,0086284907,ISDIN Corporation,Released
7,Jan 9 2023 9:02AM,254520,10,0086284906,ISDIN Corporation,Released
8,Jan 9 2023 9:02AM,254520,10,0086284911,ISDIN Corporation,Released
9,Jan 9 2023 9:02AM,254520,10,0086284912,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,254526,Released,2
15,254527,Released,1
16,254528,Released,1
17,254530,Released,2
18,254531,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Released
id,,
254526,NaN,2.0
254527,NaN,1.0
254528,NaN,1.0
254530,NaN,2.0
254531,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Released
id,,
254504,0.0,2.0
254505,2.0,9.0
254511,0.0,8.0
254512,0.0,11.0
254516,0.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,Completed,Released
id,,
254504,0,2
254505,2,9
254511,0,8
254512,0,11
254516,0,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Released
0,254504,0,2
1,254505,2,9
2,254511,0,8
3,254512,0,11
4,254516,0,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,id,Completed,Released
0,254504,,2
1,254505,2,9
2,254511,,8
3,254512,,11
4,254516,,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 9 2023 9:09AM,254527,16,"Emersa Waterbox, LLC"
1,Jan 9 2023 9:06AM,254531,10,"ClearSpec, LLC"
3,Jan 9 2023 9:03AM,254530,10,Emerginnova
5,Jan 9 2023 9:02AM,254528,10,Eye Pharma Inc
6,Jan 9 2023 9:02AM,254520,10,ISDIN Corporation
62,Jan 9 2023 8:58AM,254526,10,"Snap Medical Industries, LLC"
64,Jan 9 2023 8:58AM,254525,12,"Uniderm USA, Inc"
65,Jan 9 2023 8:55AM,254524,10,"Snap Medical Industries, LLC"
71,Jan 9 2023 8:54AM,254521,12,Hush Hush
111,Jan 9 2023 8:51AM,254522,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Released
0,Jan 9 2023 9:09AM,254527,16,"Emersa Waterbox, LLC",,1
1,Jan 9 2023 9:06AM,254531,10,"ClearSpec, LLC",,2
2,Jan 9 2023 9:03AM,254530,10,Emerginnova,,2
3,Jan 9 2023 9:02AM,254528,10,Eye Pharma Inc,,1
4,Jan 9 2023 9:02AM,254520,10,ISDIN Corporation,,56
5,Jan 9 2023 8:58AM,254526,10,"Snap Medical Industries, LLC",,2
6,Jan 9 2023 8:58AM,254525,12,"Uniderm USA, Inc",,1
7,Jan 9 2023 8:55AM,254524,10,"Snap Medical Industries, LLC",,6
8,Jan 9 2023 8:54AM,254521,12,Hush Hush,,40
9,Jan 9 2023 8:51AM,254522,12,Hush Hush,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Completed
0,Jan 9 2023 9:09AM,254527,16,"Emersa Waterbox, LLC",1,
1,Jan 9 2023 9:06AM,254531,10,"ClearSpec, LLC",2,
2,Jan 9 2023 9:03AM,254530,10,Emerginnova,2,
3,Jan 9 2023 9:02AM,254528,10,Eye Pharma Inc,1,
4,Jan 9 2023 9:02AM,254520,10,ISDIN Corporation,56,
5,Jan 9 2023 8:58AM,254526,10,"Snap Medical Industries, LLC",2,
6,Jan 9 2023 8:58AM,254525,12,"Uniderm USA, Inc",1,
7,Jan 9 2023 8:55AM,254524,10,"Snap Medical Industries, LLC",6,
8,Jan 9 2023 8:54AM,254521,12,Hush Hush,40,
9,Jan 9 2023 8:51AM,254522,12,Hush Hush,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Jan 9 2023 9:09AM,254527,16,"Emersa Waterbox, LLC",1,
1,Jan 9 2023 9:06AM,254531,10,"ClearSpec, LLC",2,
2,Jan 9 2023 9:03AM,254530,10,Emerginnova,2,
3,Jan 9 2023 9:02AM,254528,10,Eye Pharma Inc,1,
4,Jan 9 2023 9:02AM,254520,10,ISDIN Corporation,56,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet


,Date,id,Warehouse,Customer,Released,Completed
0,Jan 9 2023 9:09AM,254527,16,"Emersa Waterbox, LLC",1,NaN
1,Jan 9 2023 9:06AM,254531,10,"ClearSpec, LLC",2,NaN
2,Jan 9 2023 9:03AM,254530,10,Emerginnova,2,NaN
3,Jan 9 2023 9:02AM,254528,10,Eye Pharma Inc,1,NaN
4,Jan 9 2023 9:02AM,254520,10,ISDIN Corporation,56,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Jan 9 2023 9:09AM,254527,16,"Emersa Waterbox, LLC",1,0.0
1,Jan 9 2023 9:06AM,254531,10,"ClearSpec, LLC",2,0.0
2,Jan 9 2023 9:03AM,254530,10,Emerginnova,2,0.0
3,Jan 9 2023 9:02AM,254528,10,Eye Pharma Inc,1,0.0
4,Jan 9 2023 9:02AM,254520,10,ISDIN Corporation,56,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Completed
Warehouse,,,
10,2290684,89,2.0
12,763568,42,0.0
16,254527,1,0.0
19,509023,19,0.0
20,254517,1,0.0
22,509037,2,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Completed
0,10,2290684,89,2.0
1,12,763568,42,0.0
2,16,254527,1,0.0
3,19,509023,19,0.0
4,20,254517,1,0.0
5,22,509037,2,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Completed
0,10,89,2.0
1,12,42,0.0
2,16,1,0.0
3,19,19,0.0
4,20,1,0.0
5,22,2,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,89.0
1,12,Released,42.0
2,16,Released,1.0
3,19,Released,19.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,22
Status,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0
Released,89.0,42.0,1.0,19.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,22
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Released,89.0,42.0,1.0,19.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,22
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Released,89.0,42.0,1.0,19.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()